In [1]:
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Activation, MaxPool2D , Flatten , Dropout, TimeDistributed, MaxPooling2D, LSTM 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
#from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
import pandas as pd
import cv2
import os

import numpy as np

Using TensorFlow backend.


In [2]:
img_height , img_width = 64, 64
seq_len = 250
 
classes = ["0","1"]

In [3]:
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        #print(image)
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list

In [4]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)

     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [5]:
X, Y = create_data("video_input/")


0
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
1
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame


In [6]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size = 0.7, shuffle=False)

In [7]:
#size
print(len(X_train))
print(len(X_test))

184
79


In [9]:
model = Sequential()
model.add(TimeDistributed(Conv2D(32, 3, activation='relu', padding='same'), input_shape=(seq_len, img_height, img_width, 3)))
#model.add(TimeDistributed(Conv2D(32, 3, activation='relu', padding='same'), batch_input_shape=(10, 250, (64,64))))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),  padding='same')))

model.add(TimeDistributed(Conv2D(64, 3, activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),  padding='same' )))

model.add(TimeDistributed(Flatten()))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

model.add(LSTM(256, return_sequences=True, dropout=0.5,batch_input_shape=(10, 250, (64,64))))
model.add(LSTM(64, return_sequences=False ))


model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 250, 64, 64, 32)   896       
_________________________________________________________________
time_distributed_7 (TimeDist (None, 250, 32, 32, 32)   0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 250, 30, 30, 64)   18496     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 250, 15, 15, 64)   0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 250, 14400)        0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 250, 256)          15008768  
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)               

/Users/danielecioffi/Anaconda/anaconda3/envs/ex_cnn/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [10]:
opt = keras.optimizers.SGD(lr=0.001)
#model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])

In [11]:
earlystop = EarlyStopping(patience=5)
callbacks = [earlystop]

In [12]:
#history = model.fit(x = X_train, y = y_train, epochs=10, batch_size = 10 , shuffle=True, validation_split=0.2, callbacks=callbacks)
history = model.fit(x = X_train, y = y_train, epochs=30, batch_size = 10)
#history = model.fit(x = X_train, y = y_train, epochs=5, batch_size = 10 )

Epoch 1/30
184/184 [==============================] - 744s 4s/step - loss: 0.2270 - accuracy: 0.6793
Epoch 2/30
184/184 [==============================] - 743s 4s/step - loss: 0.2113 - accuracy: 0.7065
Epoch 3/30
184/184 [==============================] - 734s 4s/step - loss: 0.2094 - accuracy: 0.7065
Epoch 4/30
184/184 [==============================] - 761s 4s/step - loss: 0.2124 - accuracy: 0.7065
Epoch 5/30
184/184 [==============================] - 763s 4s/step - loss: 0.2085 - accuracy: 0.7065
Epoch 6/30
184/184 [==============================] - 756s 4s/step - loss: 0.2082 - accuracy: 0.7065
Epoch 7/30
184/184 [==============================] - 761s 4s/step - loss: 0.2094 - accuracy: 0.7065
Epoch 8/30
184/184 [==============================] - 763s 4s/step - loss: 0.2096 - accuracy: 0.7065
Epoch 9/30
184/184 [==============================] - 765s 4s/step - loss: 0.2079 - accuracy: 0.7065
Epoch 10/30
184/184 [==============================] - 768s 4s/step - loss: 0.2093 - accura

KeyboardInterrupt: 

In [12]:
score = model.evaluate(X_test, y_test, verbose=0, batch_size=10)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7731669493343519
Test accuracy: 0.47826087474823


In [16]:
#precition
predicted = model.predict(X_test)
print(predicted)

[[0.54291606 0.41459686]
 [0.54312253 0.42342973]
 [0.5526904  0.3964185 ]
 [0.5688049  0.37446967]
 [0.5526134  0.4056589 ]
 [0.56004506 0.39044085]
 [0.58115643 0.39528155]
 [0.54312253 0.42342973]
 [0.56266636 0.39721856]
 [0.53530747 0.41030377]
 [0.5547716  0.38984627]
 [0.540963   0.4033923 ]
 [0.56900895 0.38300696]
 [0.5319127  0.41410285]
 [0.5528961  0.4051321 ]
 [0.5688049  0.37446967]
 [0.54312253 0.42342973]
 [0.5528961  0.405132  ]
 [0.5435059  0.40896574]
 [0.54312253 0.42342973]
 [0.54312253 0.42342973]
 [0.5331898  0.3880056 ]
 [0.54312253 0.4234297 ]]


In [17]:
predicted = np.argmax(predicted, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [18]:
'''from sklearn.metrics import classification_report 
print(classification_report(y_test, predicted))
'''

              precision    recall  f1-score   support

           0       0.65      1.00      0.79        15
           1       0.00      0.00      0.00         8

    accuracy                           0.65        23
   macro avg       0.33      0.50      0.39        23
weighted avg       0.43      0.65      0.51        23



/Users/danielecioffi/Anaconda/anaconda3/envs/ex_cnn/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/danielecioffi/Anaconda/anaconda3/envs/ex_cnn/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/danielecioffi/Anaconda/anaconda3/envs/ex_cnn/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [ ]:
#prediction
pred = model.predict_classes(X_test)
print(pred)

In [20]:
for i in range(len(pred)):
    print("X_test=%s, class=%s, Predicted class=%s" % (i,y_test[i], pred[i]))

X_test=0, class=0, Predicted class=0
X_test=1, class=0, Predicted class=1
X_test=2, class=1, Predicted class=1
X_test=3, class=0, Predicted class=0
X_test=4, class=0, Predicted class=1
X_test=5, class=0, Predicted class=1
X_test=6, class=0, Predicted class=0
X_test=7, class=0, Predicted class=0
X_test=8, class=1, Predicted class=0
X_test=9, class=1, Predicted class=1
X_test=10, class=0, Predicted class=0
X_test=11, class=1, Predicted class=1
X_test=12, class=0, Predicted class=0
X_test=13, class=1, Predicted class=1
X_test=14, class=0, Predicted class=1
X_test=15, class=1, Predicted class=0
X_test=16, class=1, Predicted class=1


In [ ]:
#stampare confusion 
from sklearn.metrics import confusion_matrix
array = confusion_matrix(y_test, pred, *, labels=None, sample_weight=None, normalize=None)